In [8]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis') # 如果运行这两句命令，就会自动下载一个预训练好的模型和对呀的tokenizer
classifier("what's your name?")
#classifier("We  want to show you the 🤗 Transformers library.")



[{'label': 'NEGATIVE', 'score': 0.9468035101890564}]

In [23]:
# 在本例子中，我们需要使用的Tokenizer 和 model 
from transformers import AutoTokenizer,AutoModelForSequenceClassification

# 这个下载内容到哪里了？ 可以在https://huggingface.co/transformers/installation.html#caching-models 中
# 查看此问题的解
model_name = "bert-base-chinese"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipe = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# returns a dictionary string to list of ints
inputs = tokenizer("We are very happy to show you the 🤗 Transformers library.","What's your name")  # 直接把这句话转成了toekens
print(inputs)

output = tokenizer.tokenize("我爱你")
print(output)
print(tokenizer.convert_tokens_to_ids(output))
print(tokenizer.convert_tokens_to_ids(inputs))



Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

{'input_ids': [101, 100, 8995, 11785, 9200, 8228, 8658, 8357, 8174, 100, 100, 11616, 119, 102, 100, 112, 161, 8880, 8617, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['我', '爱', '你']
[2769, 4263, 872]
[100, 100, 100]
input_ids:[[101, 100, 8995, 11785, 9200, 8228, 8658, 8357, 8174, 100, 100, 11616, 119, 102], [101, 100, 10931, 8154, 8357, 9524, 112, 162, 11643, 8299, 8233, 119, 102, 0]]
token_type_ids:[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
attention_mask:[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]


In [29]:
"""
the tokenizer is responsible for the preprocessing of your texts.
"""
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pt_batch = tokenizer(
    # pass a list of sentences directly to your tokenizer. 但是最长不超过两个？
     ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."], 
     padding=True,
     truncation=True,
     return_tensors="pt"
 )

# 说明pt_batch 是一个dict
for key, value in pt_batch.items():
    print(f"{key}:{value.numpy().tolist()}")
pt_outputs = pt_model(**pt_batch)
# all outputs are tuples (with only one element potentially)  => 也可能是只有一个元素的元组
# 所有的Transformers 模型反馈的是尚未使用激活函数激活的值，这么做的原因是：我们通常会将激活函数和损失函数一起使用。
print(type(pt_outputs)) 
print(pt_outputs[0])# 因为是个元组，所以取其下标为0的地方



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


input_ids:[[101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], [101, 2057, 3246, 2017, 2123, 1005, 1056, 5223, 2009, 1012, 102, 0, 0, 0]]
attention_mask:[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]
<class 'tuple'>
tensor([[-4.0833,  4.3364],
        [ 0.0818, -0.0418]], grad_fn=<AddmmBackward>)


In [36]:
"""什么是Attention mask?
01.在使用句子训练模型时，各个句子很难保证长度相同。但是我们又必须用这些长度不同的句子进行训练，那么该怎么办呢？
02.处理的方法有两个：先把短的padding 至长的那么长，或者是把长的截成短的那个长度，这样就可以开始批处理，进行训练。
03.但是一般为了不想丢失信息，通常会使用第一种方法，即将短的变长。这个变长就是俗称的padding 。
那么不是经过padding得到的值对应在attention_mask的位置就是1；相反，如果是经过 padding 得到的，那么该值对应在
attention_mask 位置上的值就是0。
04.对于BertTokenizer来说，1就代表应该被“关照【即注意】”的词，而0就代表是一个padded value。

05.下面给出一个实际例子来看。
"""
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
sequence_a = "This is a short sequence."
sequence_b = "This is a rather long sequence. It is at least longer than the sequence A."
encoded_sequence_a = tokenizer(sequence_a)["input_ids"]
encoded_sequence_b = tokenizer(sequence_b)["input_ids"]
print(len(encoded_sequence_a),len(encoded_sequence_b))

#注意这里tokenizer()中的参数是一个列表
padded_sequences = tokenizer([sequence_a,sequence_b],padding=True)
#print(padded_sequences)
for key,value in padded_sequences.items():
    print(key,":",value)

8 19
input_ids : [[101, 1188, 1110, 170, 1603, 4954, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1188, 1110, 170, 1897, 1263, 4954, 119, 1135, 1110, 1120, 1655, 2039, 1190, 1103, 4954, 138, 119, 102]]
token_type_ids : [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
attention_mask : [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


In [38]:
"""Token Type IDs
一些模型的目的是做句子分类或者是问答系统。这就要求两个不同的句子被编码成同一个input IDs。 【这是什么逻辑？？】它们通常使用特殊的标记来分割，
诸如classifier 以及 分隔符。例如：BERT 模型在处理两个序列的输入时：
[....]
"""
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
sequence_a = "HuggingFace is based in NYC"
sequence_b = "Where is HuggingFace based?"
encoded_dict = tokenizer(sequence_a, sequence_b)
decoded = tokenizer.decode(encoded_dict["input_ids"])

print(decoded) 
# 上面的这个输出可能对一般的模型已经足够好了。因为一个句子的SEP 就表示着 另一个句子的start。但是BERT却有另外一个机制 —— token type IDs
# 同时也被叫做 segment IDs。它们用于区分模型中不同的序列。
print(encoded_dict['token_type_ids'])

[CLS] HuggingFace is based in NYC [SEP] Where is HuggingFace based? [SEP]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
